In [1]:
import gensim

In [2]:
import json
clean_data = json.load(open('data.json'))

In [3]:
titles = list(clean_data.keys())

In [4]:
train = []
for title in titles:
  docs = clean_data[title]
  dictionary = gensim.corpora.Dictionary(docs)
  bow_corpus = [dictionary.doc2bow(doc) for doc in docs]
  lda_model = gensim.models.ldamodel.LdaModel(bow_corpus, 
                                   num_topics = 1, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   )
  train.append((' '.join(lda_model.print_topics(-1)[0][1].split('"')[1::2]), title))

In [5]:
import pandas as pd
train_df = pd.DataFrame(train, columns=['input_text', 'target_text'])

In [6]:
!pip install simpletransformers
!pip install transformers

     |████████████████████████████████| 221 kB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 82.3 MB/s 
     |████████████████████████████████| 1.6 MB 74.6 MB/s 
     |████████████████████████████████| 2.6 MB 68.8 MB/s 
     |████████████████████████████████| 8.0 MB 24.3 MB/s 
     |████████████████████████████████| 1.2 MB 68.2 MB/s 
     |████████████████████████████████| 124 kB 63.2 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 264 kB 84.5 MB/s 
     |████████████████████████████████| 636 kB 73.4 MB/s 
     |████████████████████████████████| 895 kB 82.2 MB/s 
     |████████████████████████████████| 97 kB 7.6 MB/s 
     |████████████████████████████████| 133 kB 78.0 MB/s 
     |████████████████████████████████| 170 kB 90.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 118 kB 79.4 MB/s 
     |████████████████████████████████| 243 kB 88.6 MB/s 
     |███████████████

In [51]:
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_seq_length = 64
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
#model_args.wandb_project = "Paraphrasing with BART"


model = Seq2SeqModel(
    encoder_decoder_type="mbart",
    encoder_decoder_name="IlyaGusev/mbart_ru_sum_gazeta",
    args=model_args, use_cuda=False
)

In [50]:
import shutil
shutil.rmtree('cache_dir')

In [ ]:
model.train_model(train_df.sample(frac=1))

  0%|          | 0/164 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3351: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
pred = model.predict(list(train_df['input_text']))

In [44]:
pred[:6]

[['Вакцина от COVID-19', 'Вакцина от COVID-19', 'Вакцина от COVID-19'],
 ['Политический кризис в Молдавии',
  'Взрыв дипломатических переговоров в Молдавии',
  'Взрыв дипломатической линии Молдавии'],
 ['Взрыв спутника ГЛОНАСС', 'На орбиту спутников', 'Вакцина от ракеты SpaceX'],
 ['Вакцина ММВБ', 'Вакцина ММВБ', 'Вакцина ЦБ РФ'],
 ['Заседание Московской биржи', 'Внешние мировые индексы', 'Вакцина на бирже'],
 ['Вакцина «Атомстрой"',
  'Соглашение о сделке ОПЕК+ и ОПЕК+',
  'Вакцина Сбербанка']]